In [1]:
#installing dependencies
import sqlite3
import pandas as pd  
dbfile = '/Users/sofyakonchakova/Desktop/Data_Science/data/home-assistant_v2.db'


### SQL Data Pulling


In [2]:
#with these lines access the sql database
con = sqlite3.connect(dbfile)
cur = con.cursor()
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
print(table_list)
con.close()

[('events',), ('recorder_runs',), ('schema_changes',), ('states',), ('statistics_meta',), ('statistics',), ('statistics_short_term',), ('statistics_runs',)]


In [3]:
#accessing the events data base
with sqlite3.connect(dbfile) as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM events")
    events = cur.fetchall()

#converting it to the pandas data frame
from_events_db = []

for result in events:
    result = list(result)
    from_events_db.append(result)

columns = ["event_id","event_type","event_data","origin","time_fired","created","context_id","context_user_id","context_parent_id"]
events_df = pd.DataFrame(from_events_db, columns = columns)

In [4]:
#example of exported lines
for x in events[0:5]:
    print(x)

(11287622, 'state_changed', '{}', 'LOCAL', '2021-12-02 03:12:05.162304', '2021-12-02 03:12:05.162304', '165b40ded15eb3ac829cb9cc928488e0', None, None)
(11287623, 'state_changed', '{}', 'LOCAL', '2021-12-02 03:12:05.162807', '2021-12-02 03:12:05.162807', '96e82b80f0ac922bccb9cc3cec0a0cd1', None, None)
(11287624, 'state_changed', '{}', 'LOCAL', '2021-12-02 03:12:05.191776', '2021-12-02 03:12:05.191776', 'b9147b7388f02ee17b913d72199a6c86', None, None)
(11287625, 'state_changed', '{}', 'LOCAL', '2021-12-02 03:12:05.220377', '2021-12-02 03:12:05.220377', '13a83bcf01322ee87c31abe3d6956df9', None, None)
(11287626, 'state_changed', '{}', 'LOCAL', '2021-12-02 03:12:05.252402', '2021-12-02 03:12:05.252402', '6304dddf2693fa55a72c5d39c39f77be', None, None)


In [5]:
#acessing the schema_changes database
with sqlite3.connect(dbfile) as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM schema_changes")
    schema_changes = cur.fetchall()

#converting it to the pandas data frame
from_schema_changes_db = []

for result in schema_changes:
    result = list(result)
    from_schema_changes_db.append(result)

columns = ["change_id","schema_version","changed"]
schema_changes_df = pd.DataFrame(from_schema_changes_db, columns = columns)

In [6]:
#acessing the states database
with sqlite3.connect(dbfile) as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM states")
    states = cur.fetchall()

    
#converting it to the pandas data frame
from_states_db = []

for result in states:
    result = list(result)
    from_states_db.append(result)

columns = ["state_id","domain","entity_id","state","attributes","event_id","last_changed","last_updated","created","old_state_id"]
states_df = pd.DataFrame(from_states_db, columns = columns)

In [7]:
#acessing the statistics database
with sqlite3.connect(dbfile) as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM statistics")
    statistics = cur.fetchall()

#converting it to the pandas data frame
from_statistics_db = []

for result in statistics:
    result = list(result)
    from_statistics_db.append(result)

columns = ["id","created","metadata_id","start","mean","min","max","last_reset","state","sum"]
statistics_df = pd.DataFrame(from_statistics_db, columns = columns)

In [8]:
#acessing the statistics_meta database
with sqlite3.connect(dbfile) as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM statistics_meta")
    statistics_meta = cur.fetchall()

#converting it to the pandas data frame
from_statistics_meta_db = []

for result in statistics_meta:
    result = list(result)
    from_statistics_meta_db.append(result)

columns = ["id","statistic_id","source","unit_of_measurement","has_mean","has_sum","name"]
statistics_meta_df = pd.DataFrame(from_statistics_meta_db, columns = columns)

In [9]:
#acessing the statistics_runs database
with sqlite3.connect(dbfile) as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM statistics_runs")
    statistics_runs = cur.fetchall()

#converting it to the pandas data frame
from_statistics_runs_db = []

for result in statistics_runs:
    result = list(result)
    from_statistics_runs_db.append(result)

columns = ["run_id","start"]
statistics_runs_df = pd.DataFrame(from_statistics_runs_db, columns = columns)

In [10]:
#acessing the statistics_short database
with sqlite3.connect(dbfile) as con:
    cur = con.cursor()
    cur.execute("SELECT * FROM statistics_short_term")
    statistics_short_term = cur.fetchall()

#converting it to the pandas data frame
from_statistics_short_term_db = []

for result in statistics_short_term:
    result = list(result)
    from_statistics_short_term_db.append(result)

columns = ["id","created","start","mean","min","max","last_reset","state","sum","metadata_id"]
statistics_short_term_df = pd.DataFrame(from_statistics_short_term_db, columns = columns)

In [11]:
#shapes of these dataframes

print(f'The shape of datasets: \n events: {events_df.shape}, \n schema_changes : {schema_changes_df.shape}\n states : {states_df.shape}, \n statistics : {statistics_df.shape},\n statistics_meta: {statistics_meta_df.shape},\n statistics_runs : {statistics_runs_df.shape},\n statistics_short_term: {statistics_runs_df.shape}')

The shape of datasets: 
 events: (508630, 9), 
 schema_changes : (13, 3)
 states : (507505, 10), 
 statistics : (12340, 10),
 statistics_meta: (13, 7),
 statistics_runs : (2954, 2),
 statistics_short_term: (2954, 2)


In [12]:
states_df.head()

,state_id,domain,entity_id,state,attributes,event_id,last_changed,last_updated,created,old_state_id
0,11244860,zwave,zwave.switch_cr,ready,"{""node_id"":15,""node_name"":""Shenzhen Neo Electr...",11287622,2021-11-28 20:46:35.720593,2021-12-02 03:12:05.162304,2021-12-02 03:12:05.162304,NaN
1,11244861,sensor,sensor.switch_cr_exporting,False,"{""node_id"":15,""value_index"":32,""value_instance...",11287623,2021-12-02 03:12:05.162807,2021-12-02 03:12:05.162807,2021-12-02 03:12:05.162807,NaN
2,11244862,sensor,sensor.switch_cr_energy,6.86,"{""node_id"":15,""value_index"":0,""value_instance""...",11287624,2021-12-02 03:12:05.191776,2021-12-02 03:12:05.191776,2021-12-02 03:12:05.191776,NaN
3,11244863,sensor,sensor.switch_cr_previous_reading_1,6.86,"{""node_id"":15,""value_index"":1,""value_instance""...",11287625,2021-12-02 03:12:05.220377,2021-12-02 03:12:05.220377,2021-12-02 03:12:05.220377,NaN
4,11244864,sensor,sensor.switch_cr_interval_1,301,"{""node_id"":15,""value_index"":2,""value_instance""...",11287626,2021-12-02 03:12:05.252402,2021-12-02 03:12:05.252402,2021-12-02 03:12:05.252402,NaN


### Preparation Agent

In [19]:
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507505 entries, 0 to 507504
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   state_id      507505 non-null  int64  
 1   domain        507505 non-null  object 
 2   entity_id     507505 non-null  object 
 3   state         507504 non-null  object 
 4   attributes    507505 non-null  object 
 5   event_id      507505 non-null  int64  
 6   last_changed  507505 non-null  object 
 7   last_updated  507505 non-null  object 
 8   created       507505 non-null  object 
 9   old_state_id  506724 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 38.7+ MB


In [20]:
df = states_df.copy()

In [21]:
df.head()

,state_id,domain,entity_id,state,attributes,event_id,last_changed,last_updated,created,old_state_id
0,11244860,zwave,zwave.switch_cr,ready,"{""node_id"":15,""node_name"":""Shenzhen Neo Electr...",11287622,2021-11-28 20:46:35.720593,2021-12-02 03:12:05.162304,2021-12-02 03:12:05.162304,NaN
1,11244861,sensor,sensor.switch_cr_exporting,False,"{""node_id"":15,""value_index"":32,""value_instance...",11287623,2021-12-02 03:12:05.162807,2021-12-02 03:12:05.162807,2021-12-02 03:12:05.162807,NaN
2,11244862,sensor,sensor.switch_cr_energy,6.86,"{""node_id"":15,""value_index"":0,""value_instance""...",11287624,2021-12-02 03:12:05.191776,2021-12-02 03:12:05.191776,2021-12-02 03:12:05.191776,NaN
3,11244863,sensor,sensor.switch_cr_previous_reading_1,6.86,"{""node_id"":15,""value_index"":1,""value_instance""...",11287625,2021-12-02 03:12:05.220377,2021-12-02 03:12:05.220377,2021-12-02 03:12:05.220377,NaN
4,11244864,sensor,sensor.switch_cr_interval_1,301,"{""node_id"":15,""value_index"":2,""value_instance""...",11287626,2021-12-02 03:12:05.252402,2021-12-02 03:12:05.252402,2021-12-02 03:12:05.252402,NaN


In [22]:
df['attributes'] = df['attributes'].apply(lambda x: x.replace('true','True'))
df['attributes'] = df['attributes'].apply(lambda x: x.replace('false','False'))
df['attributes'] = df['attributes'].apply(lambda x: x.replace('null','None'))

In [23]:
df['attributes']=df['attributes'].apply(lambda dat: dict(eval(dat)))
df2 = pd.json_normalize(df['attributes'])
result = pd.concat([df,df2], axis = 1).drop('attributes', axis = 1)

In [24]:
result.created.min()

'2021-12-02 03:12:05.162304'

In [25]:
result.created.max()

'2021-12-12 09:28:16.840038'

Minimum Viable Product: define the list we need( )

In [40]:
trial = result.copy()
trial = trial.dropna(axis = 1, thresh =100000)

In [41]:
trial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507505 entries, 0 to 507504
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   state_id             507505 non-null  int64  
 1   domain               507505 non-null  object 
 2   entity_id            507505 non-null  object 
 3   state                507504 non-null  object 
 4   event_id             507505 non-null  int64  
 5   last_changed         507505 non-null  object 
 6   last_updated         507505 non-null  object 
 7   created              507505 non-null  object 
 8   old_state_id         506724 non-null  float64
 9   node_id              474497 non-null  float64
 10  friendly_name        507503 non-null  object 
 11  value_index          388280 non-null  float64
 12  value_instance       388280 non-null  float64
 13  value_id             388280 non-null  object 
 14  power_consumption    251940 non-null  float64
 15  unit_of_measureme

In [42]:
trial.head()

,state_id,domain,entity_id,state,event_id,last_changed,last_updated,created,old_state_id,node_id,friendly_name,value_index,value_instance,value_id,power_consumption,unit_of_measurement
0,11244860,zwave,zwave.switch_cr,ready,11287622,2021-11-28 20:46:35.720593,2021-12-02 03:12:05.162304,2021-12-02 03:12:05.162304,NaN,15.0,Switch,NaN,NaN,NaN,NaN,NaN
1,11244861,sensor,sensor.switch_cr_exporting,False,11287623,2021-12-02 03:12:05.162807,2021-12-02 03:12:05.162807,2021-12-02 03:12:05.162807,NaN,15.0,Switch Exporting,32.0,1.0,72057594294600192,0.0,
2,11244862,sensor,sensor.switch_cr_energy,6.86,11287624,2021-12-02 03:12:05.191776,2021-12-02 03:12:05.191776,2021-12-02 03:12:05.191776,NaN,15.0,Switch Energy,0.0,1.0,72057594294599682,0.0,kWh
3,11244863,sensor,sensor.switch_cr_previous_reading_1,6.86,11287625,2021-12-02 03:12:05.220377,2021-12-02 03:12:05.220377,2021-12-02 03:12:05.220377,NaN,15.0,Switch Previous Reading 1,1.0,1.0,72057594294599698,0.0,kWh
4,11244864,sensor,sensor.switch_cr_interval_1,301,11287626,2021-12-02 03:12:05.252402,2021-12-02 03:12:05.252402,2021-12-02 03:12:05.252402,NaN,15.0,Switch Interval 1,2.0,1.0,72057594294599715,0.0,seconds


In [43]:
trial.power_consumption.unique()

array([  nan,  0.  , 27.2 , 26.46, 26.28, 26.84, 27.11, 26.61, 26.86,
       26.06, 25.97, 26.12,  4.33, 26.01,  4.42, 25.98,  4.47, 26.09,
        4.31, 26.02,  4.38, 26.47,  4.4 , 26.  , 26.55,  4.3 , 26.5 ,
        4.43, 26.6 ,  4.34,  4.36,  4.29,  4.35,  4.37,  4.27,  4.14,
        4.24,  4.1 ,  4.04,  4.06, 27.25,  4.08,  4.15,  4.03,  4.05,
        4.39,  4.41,  4.48,  4.45,  4.12,  4.01,  4.44,  4.5 ,  4.32,
        4.28,  4.26, 13.96, 13.86, 13.91, 13.83, 13.43, 13.54, 13.65,
       13.72, 13.32, 13.35, 11.18,  5.18,  4.02,  3.97,  3.99,  4.09,
        3.95,  4.  ,  3.98,  3.94,  3.96,  4.07,  3.92,  3.89,  3.93,
        3.42,  4.2 ,  4.17,  4.23,  4.13,  4.16,  4.19,  4.22,  4.21,
        4.25,  4.11,  4.18])

In [85]:
kwh_data = trial[trial.unit_of_measurement.isin(['kWh']) & trial.node_id.isin([3,15,34,46])]

In [79]:
kwh_data_long = kwh_data[['node_id','value_id','last_changed','last_updated', 'power_consumption']]
kwh_data_wide = pd.pivot(kwh_data_long, index = ['last_changed'], columns = 'node_id', values = 'power_consumption')
kwh_data_wide

node_id,3.0,15.0,34.0,46.0
last_changed,,,,
2021-12-02 03:12:05.295427,NaN,0.0,NaN,NaN
2021-12-02 03:12:05.295772,NaN,0.0,NaN,NaN
2021-12-02 03:14:29.047935,NaN,NaN,NaN,0.0
2021-12-02 03:14:29.048387,NaN,NaN,NaN,0.0
2021-12-02 03:15:04.523666,0.0,NaN,NaN,NaN
...,...,...,...,...
2021-12-12 09:22:27.612285,NaN,NaN,0.0,NaN
2021-12-12 09:25:12.697801,0.0,NaN,NaN,NaN
2021-12-12 09:25:12.698170,0.0,NaN,NaN,NaN


In [80]:
kwh_data_wide.isna().sum()

node_id
3.0     13214
15.0    13757
34.0    12540
46.0    18386
dtype: int64

In [81]:
#filling nas with zeros 
kwd_data_wide = kwh_data_wide.fillna(0)
df = kwd_data_wide.reset_index()

In [83]:
#aggregating per hour
df.last_changed = pd.to_datetime(df.last_changed)
df = df.resample('60T', on="last_changed").sum()

In [84]:
#two nodes out of use?
df.sum()

node_id
3.0     3016.48
15.0       0.00
34.0       0.00
46.0    1426.82
dtype: float64

3 - kitchen lamp 
15 - child room lamp
34 - bedroom lamp
46 - living room
41 - 
48 - rolls for windows, disregard


-> aggregate per hour


-> focus on integration